This notebook computes all of the "distances" (the fuzzy logic scores for IOU, DICE, ...) between the GradCam maps produced by the final models and the visual characterstics maps created by the dermatologists. That is: 

```
for each image i  
  for each GradCam image i_gc  
    for each dermatologist d  
      for each characteristic d_char  
        compute the visual distance between i_gc and d_char 
```


In [122]:
%matplotlib inline
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt 
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [39]:
classes = [
    "Acne",
    "Actinic keratosis",
    "Psoriasis",
    "Seborrheic dermatitis",
    "Viral warts",
    "Vitiligo"
]

derms = [
    'derm1',
    'derm2',
    'derm3',
    'derm4',
    'derm5',
    'derm6',
    'derm7',
    'derm8',
]

characteristics = [
    'Closed comedo',
    'Cyst',
    'Dermatoglyph disruption',
    'Leukotrichia',
    'Macule',
    'Nodule',
    'Open comedo',
    'Papule',
    'Patch',
    'Plaque',
    'Pustule',
    'Scale',
    'Scar',
    'Sun damage',
    'Telangiectasia',
    'Thrombosed capillaries'
]

images_path = Path('/home/ubuntu/hot-store/dermx/all_images')
masks_path = Path('/home/ubuntu/hot-store/dermx_masks/per_image')
results_path = Path('/home/ubuntu/hot-store/final/')
columns = [
         'filename',
         'gradcam_class',
         'iou',
         'dice',
         'precision',
         'recall',
         'negative_predictive_value',
         'specificity'
    ]


# Metrics supporting probabilistic segmentation maps
fuzzy_and = lambda x,y: np.minimum(x,y)
fuzzy_or = lambda x,y: np.maximum(x,y)
fuzzy_not = lambda x: 1-x

def pixel_metrics_fuzzy(y_true, y_pred):
    """
    Pixel-level metrics of segmentation accuracy following fuzzy logic operators.
    
    :param y_true: numpy.ndarray of reference segmentation, values in [0,1]
    :param y_pred: numpy.ndarray of predicted segmentation, values in [0,1]

    :return: a dictionary encoding the metrics
    """
        
    np.testing.assert_equal(y_true.shape, y_pred.shape, err_msg="Expecting \
    the reference and predicted segmentations to be of the same size.")
    
    # Check the ranges
    np.testing.assert_equal(np.logical_and(y_true >= 0, y_true <= 1).all(), True, err_msg="Expecting \
    the reference segmentations to be in the range 0 to 1.")
    np.testing.assert_equal(np.logical_and(y_pred >= 0, y_pred <= 1).all(), True, err_msg="Expecting \
    the predicted segmentations to be in the range 0 to 1.")
    
    TP = fuzzy_and(y_true, y_pred).sum()
    TN = fuzzy_and(fuzzy_not(y_true), fuzzy_not(y_pred)).sum()
    union = fuzzy_or(y_true, y_pred).sum()
    
    metrics = {}
    
    # Summary metrics
    metrics["iou"] = TP / union
    metrics["dice"] = 2 * TP / ( y_true.sum() + y_pred.sum() ) 
    
    # Positive class metrics
    metrics["precision"] = TP / y_pred.sum()
    metrics["recall"] = TP / y_true.sum()
    
    # Negative class metrics
    metrics["negative_predictive_value"] = TN / fuzzy_not(y_pred).sum()
    metrics["specificity"] = TN / fuzzy_not(y_true).sum()
    
    return metrics


def calculate_mask_metrics(gradcam_path, mask_path, interpolation_method=Image.NEAREST):
    """
    Calculates the fuzzy logic metrics given the paths to a pair of input images.
    The derm mask is resized to match the size of the gradcam image.
    
    Input:
    - gradcam_image_path: Pathlib path to a gradCam image. The file is assumed to be in .npy format.
    - derm_char_mask_path: Pathlib path to a derm annotation. The file is assumed to be in a format that can
                           be opened by PIL.
    - interpolation_method: String. The method used for interpolation when resizing the derm mask. Options are
                            NEAREST, BOX, BILINEAR, HAMMING, BICUBIC, LANCZOS. Default is NEAREST.    
    """
    
    # Open images.
    gradcam = np.load(gradcam_path, allow_pickle=True)
    mask = Image.open(mask_path)
    
    # Resize the derm mask if its size does not match the size of the gradcam image.
    if gradcam.shape != mask.size[::-1]:
        # Note that resize uses (cols, rows) format, while .shape is in (rows, cols) format.
        mask = mask.resize((gradcam.shape[::-1]), interpolation_method)
    
    # Convert the derm mask to numpy format and normalize to [0, 1].
    mask = np.asarray(mask) / 255

    return pixel_metrics_fuzzy(mask, gradcam)


## Per image metrics

In [22]:
def get_per_image_metrics(results_path, model_name, images_path, masks_path, classes, columns):
    gradcams_path = results_path / 'visualisation/gradcam' / model_name
    model_path = results_path / model_name
    
    derm_mask_paths = [mask_path for mask_path in masks_path.iterdir() if mask_path.suffix == '.png']
    image_paths = [image_path for image_path in images_path.rglob('*.jpeg')]

    # For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
    # class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
    # As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
    metrics_list = []

    # Set the method used for interpolation when resizing the derm annotations.
    interpolation = Image.NEAREST

    # Compute metrics for each image
    for image_path in image_paths:
        for diagnosis in classes:
            # Check if matching GradCam file exists.
            gradcam_path = gradcams_path / Path(image_path.stem + '_' + diagnosis + '.npy')
            if gradcam_path.is_file():
                mask_path = masks_path / Path(image_path.stem + '.png') 
                if mask_path.is_file():
                    # Calculate the value of the metics given the GradCam image and the derm mask.
                    gradcam_metric_val = calculate_mask_metrics(gradcam_path, mask_path, interpolation)
                    gradcam_metric_val['gradcam_class'] = diagnosis 
                    gradcam_metric_val['filename'] = image_path.stem 
                    metrics_list.append(gradcam_metric_val)
            else:
                print(f'GradCam file missing for image {image_path}')
                
    return pd.DataFrame.from_records(metrics_list, columns=columns)


In [23]:
def analyse_per_image_gradcam(
    results_path, 
    model_name, 
    images_path, 
    masks_path, 
    preds_name,
    columns,
    classes,
):
    per_image_metrics_df = get_per_image_metrics(results_path, model_name, images_path, masks_path, classes, columns)
    
    preds_df = pd.read_csv(results_path / preds_name)
    preds_df['filename'] = [filename.split('/')[1].split('.')[0] for filename in preds_df.filename.values]
    preds_df = preds_df.merge(per_image_metrics_df, left_on='filename', right_on='filename')
    # Keep only correct predictions
    preds_df = preds_df[preds_df['actual'] == preds_df['pred']]
    preds_df = preds_df[preds_df['pred_class'] == preds_df['gradcam_class']]
    
    return {
        'iou': preds_df['iou'].mean(),
        'dice': preds_df['dice'].mean(),
        'precision': preds_df['precision'].mean(),
        'recall': preds_df['recall'].mean(),
        'negative_predictive_value': preds_df['negative_predictive_value'].mean(),
        'specificity': preds_df['specificity'].mean(), 
    }


In [25]:
model_info = [
    {
        'model_name': 'efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_',
        'model_architecture': 'efficientnet',
    },
    {
        'model_name': 'inception_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.001_lactivation_85_',
        'model_architecture': 'inception',
    },
    {
        'model_name': 'inceptionresnetv2_r20_s0.25_z0.5_b[0.75, 1.25]_lr0.0001_lblock8_9_ac_',
        'model_architecture': 'inceptionresnet',
    },
    {
        'model_name': 'mobilenetv1_r10_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv_pw_12_relu_',
        'model_architecture': 'mobilenetv1',
    },
    {
        'model_name': 'mobilenetv2_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock_15_add_',
        'model_architecture': 'mobilenetv2',
    },
    {
        'model_name': 'nasnetmobile_r20_s0.25_z0.5_b[0.5, 1.0]_lr0.0001_lnormal_concat_11_',
        'model_architecture': 'nasnetmobile',
    },
    {
        'model_name': 'resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_',
        'model_architecture': 'resnet',
    },
    {
        'model_name': 'resnetv2_r20_s0.25_z0.25_b[0.5, 1.0]_lr0.001_lpost_relu_',
        'model_architecture': 'resnetv2',
    },
    {
        'model_name': 'vgg_r10_s0.0_z0.25_b[0.5, 1.0]_lr0.01_lblock5_pool_',
        'model_architecture': 'vgg',
    },
    {
        'model_name': 'xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_', 
        'model_architecture': 'xception',
    }
]

models_performance = []
for model in model_info:
    for idx in range(5):
        model_results_path = results_path / model['model_architecture']
        model_name = f'{model["model_name"]}{idx}'
        preds_name = f'dermx_{idx}_preds.csv'
        print(model_name)
        model_performance = analyse_per_image_gradcam(model_results_path, model_name, images_path, masks_path, preds_name, columns, classes)
        model_performance['model_architecture'] = model['model_architecture']
        model_performance['model_name'] = model['model_name']
        models_performance.append(model_performance)
models_performance_df = pd.DataFrame.from_records(models_performance, columns=models_performance[0].keys())   

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_0


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_1


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_2


...

/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_3


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_4


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


In [29]:
models_performance_df.groupby('model_architecture').describe()

iou                                                    \
                   count      mean       std       min       25%       50%   
model_architecture                                                           
efficientnet         5.0  0.271129  0.007888  0.261790  0.266514  0.270400   
inception            5.0  0.286989  0.003142  0.282876  0.285292  0.286697   
inceptionresnet      5.0  0.233090  0.004154  0.228076  0.231333  0.231344   
mobilenetv1          5.0  0.248339  0.004474  0.243787  0.245216  0.246927   
mobilenetv2          5.0  0.265011  0.012498  0.254713  0.256548  0.262732   
nasnetmobile         5.0  0.298781  0.004685  0.292928  0.297742  0.298184   
resnet               5.0  0.193838  0.017544  0.179002  0.180388  0.183897   
resnetv2             5.0  0.264114  0.005328  0.257162  0.261158  0.263866   
vgg                  5.0  0.201695  0.038193  0.150934  0.177983  0.202910   
xception             5.0  0.298148  0.005708  0.291791  0.292621  0.300193   

                                        dice                                \
                         75%       max count      mean       std       min   
model_architecture                                                           
efficientnet        0.274468  0.282474   5.0  0.405789  0.010088  0.393204   
inception           0.289334  0.290746   5.0  0.429647  0.002934  0.426046   
inceptionresnet     0.236485  0.238211   5.0  0.361777  0.005437  0.355904   
mobilenetv1         0.251104  0.254660   5.0  0.382329  0.005918  0.376158   
mobilenetv2         0.265028  0.286033   5.0  0.402131  0.015731  0.389625   
nasnetmobile        0.299056  0.305993   5.0  0.440660  0.006486  0.432673   
resnet              0.212342  0.213560   5.0  0.303606  0.023640  0.282943   
resnetv2            0.267632  0.270751   5.0  0.404987  0.006501  0.396921   
vgg                 0.236008  0.240643   5.0  0.321106  0.051870  0.251019   
xception            0.301225  0.304908   5.0  0.442962  0.006920  0.435429   

                                                           precision  \
                         25%       50%       75%       max     count   
model_architecture                                                     
efficientnet        0.400478  0.406024  0.408946  0.420297       5.0   
inception           0.427899  0.429821  0.430654  0.433814       5.0   
inceptionresnet     0.358473  0.359446  0.366745  0.368314       5.0   
mobilenetv1         0.377497  0.381438  0.386385  0.390165       5.0   
mobilenetv2         0.391474  0.399249  0.401506  0.428799       5.0   
nasnetmobile        0.439015  0.439395  0.441593  0.450625       5.0   
resnet              0.286064  0.290436  0.327363  0.331224       5.0   
resnetv2            0.400610  0.404829  0.409638  0.412935       5.0   
vgg                 0.289337  0.324886  0.367231  0.373055       5.0   
xception            0.435994  0.445625  0.446744  0.451016       5.0   

                                                                      \
                        mean       std       min       25%       50%   
model_architecture                                                     
efficientnet        0.393269  0.009688  0.381409  0.387200  0.392268   
inception           0.428529  0.008980  0.413634  0.426444  0.433896   
inceptionresnet     0.418676  0.008283  0.406971  0.414296  0.420061   
mobilenetv1         0.381843  0.007174  0.374786  0.376579  0.378802   
mobilenetv2         0.443537  0.019976  0.430108  0.433441  0.433913   
nasnetmobile        0.417228  0.009942  0.405413  0.408077  0.420157   
resnet              0.348565  0.009366  0.337207  0.342735  0.347249   
resnetv2            0.505571  0.007547  0.494863  0.500518  0.509497   
vgg                 0.465579  0.020630  0.450675  0.450880  0.459825   
xception            0.487938  0.007308  0.477525  0.484077  0.490250   

                                       recall                                \
                         75%       max

## Per characteristic metrics

In [47]:
def get_per_characteristic_metrics(results_path, model_name, images_path, masks_path, classes, columns, characteristics):
    gradcams_path = results_path / 'visualisation/gradcam' / model_name
    model_path = results_path / model_name
    
    derm_mask_paths = [mask_path for mask_path in masks_path.iterdir() if mask_path.suffix == '.png']
    image_paths = [image_path for image_path in images_path.rglob('*.jpeg')]

    # For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
    # class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
    # As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
    metrics_list = []

    # Set the method used for interpolation when resizing the derm annotations.
    interpolation = Image.NEAREST

    # Compute metrics for each image
    for image_path in image_paths:
        for characteristic in characteristics:
            for diagnosis in classes:
                # Check if matching GradCam file exists.
                gradcam_path = gradcams_path / Path(image_path.stem + '_' + diagnosis + '.npy')
                if gradcam_path.is_file():
                    mask_path = masks_path / Path(f'{characteristic}_{image_path.stem}.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calculate_mask_metrics(gradcam_path, mask_path, interpolation)
                        gradcam_metric_val['gradcam_class'] = diagnosis 
                        gradcam_metric_val['characteristic'] = characteristic 
                        gradcam_metric_val['filename'] = image_path.stem 
                        metrics_list.append(gradcam_metric_val)
                else:
                    print(f'GradCam file missing for image {image_path}')
                
    return pd.DataFrame.from_records(metrics_list, columns=columns)


In [96]:
def analyse_per_characteristics_gradcam(
    results_path, 
    model_name, 
    images_path, 
    masks_path, 
    preds_name,
    columns,
    classes,
    characteristics,
    metrics
):
    per_image_metrics_df = get_per_characteristic_metrics(results_path, model_name, images_path, masks_path, classes, columns, characteristics)
    preds_df = pd.read_csv(results_path / preds_name)
    preds_df['filename'] = [filename.split('/')[1].split('.')[0] for filename in preds_df.filename.values]
    preds_df = preds_df.merge(per_image_metrics_df, left_on='filename', right_on='filename')
    # Keep only correct predictions
    preds_df = preds_df[preds_df['actual'] == preds_df['pred']]
    preds_df = preds_df[preds_df['pred_class'] == preds_df['gradcam_class']]

    metrics_dict = {}
    for characteristic in characteristics:
        metrics_dict[characteristic] = {}
        for metric in metrics:
            metrics_dict[characteristic][metric] = preds_df[preds_df['characteristic'] == characteristic][metric].mean()
            
    return metrics_dict


In [120]:
model_info = [
    {
        'model_name': 'efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_',
        'model_architecture': 'efficientnet',
    },
    {
        'model_name': 'inception_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.001_lactivation_85_',
        'model_architecture': 'inception',
    },
    {
        'model_name': 'inceptionresnetv2_r20_s0.25_z0.5_b[0.75, 1.25]_lr0.0001_lblock8_9_ac_',
        'model_architecture': 'inceptionresnet',
    },
    {
        'model_name': 'mobilenetv1_r10_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv_pw_12_relu_',
        'model_architecture': 'mobilenetv1',
    },
    {
        'model_name': 'mobilenetv2_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock_15_add_',
        'model_architecture': 'mobilenetv2',
    },
    {
        'model_name': 'nasnetmobile_r20_s0.25_z0.5_b[0.5, 1.0]_lr0.0001_lnormal_concat_11_',
        'model_architecture': 'nasnetmobile',
    },
    {
        'model_name': 'resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_',
        'model_architecture': 'resnet',
    },
    {
        'model_name': 'resnetv2_r20_s0.25_z0.25_b[0.5, 1.0]_lr0.001_lpost_relu_',
        'model_architecture': 'resnetv2',
    },
    {
        'model_name': 'vgg_r10_s0.0_z0.25_b[0.5, 1.0]_lr0.01_lblock5_pool_',
        'model_architecture': 'vgg',
    },
    {
        'model_name': 'xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_', 
        'model_architecture': 'xception',
    }
]
masks_path = Path('/home/ubuntu/hot-store/dermx_masks/per_characteristic')
columns = [
    'filename',
    'gradcam_class',
    'characteristic',
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]
metrics = [
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]

models_performance_dict = {}
for model in model_info:
    for idx in range(5):
        model_results_path = results_path / model['model_architecture']
        model_name = f'{model["model_name"]}{idx}'
        preds_name = f'dermx_{idx}_preds.csv'
        print(model_name)
        model_performance = analyse_per_characteristics_gradcam(model_results_path, model_name, images_path, masks_path, preds_name, columns, classes, characteristics, metrics)
        for k, v in model_performance.items():
            models_performance_dict[(model['model_architecture'], model_name, k)] = v

models_performance_df = pd.DataFrame(models_performance_dict).T

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_0


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_1


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_2


...

/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_3


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_4


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


In [123]:
models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe()

iou                                \
                                        count      mean       std       min   
level_0         level_2                                                       
efficientnet    Closed comedo             5.0  0.131268  0.007443  0.123977   
                Cyst                      5.0  0.020709  0.002862  0.017472   
                Dermatoglyph disruption   0.0       NaN       NaN       NaN   
                Leukotrichia              5.0  0.084602  0.012101  0.069894   
                Macule                    5.0  0.123511  0.009233  0.110574   
                Nodule                    5.0  0.036670  0.003550  0.032114   
                Open comedo               5.0  0.123508  0.003335  0.118555   
                Papule                    5.0  0.100700  0.006568  0.092858   
                Patch                     5.0  0.140935  0.005029  0.134103   
                Plaque                    5.0  0.202421  0.004258  0.197724   
                Pustule                   5.0  0.022621  0.001745  0.020462   
                Scale                     5.0  0.189197  0.003653  0.183822   
                Scar                      5.0  0.202895  0.007959  0.194437   
                Sun damage                5.0  0.259614  0.017451  0.232212   
                Telangiectasia            5.0  0.110398  0.002795  0.107020   
                Thrombosed capillaries    0.0       NaN       NaN       NaN   
inception       Closed comedo             5.0  0.130661  0.006585  0.123488   
                Cyst                      5.0  0.017491  0.000928  0.016216   
                Dermatoglyph disruption   0.0       NaN       NaN       NaN   
                Leukotrichia              2.0  0.062269  0.038421  0.035101   
                Macule                    5.0  0.132403  0.004065  0.125922   
                Nodule                    5.0  0.040702  0.004226  0.034078   
                Open comedo               5.0  0.125052  0.010055  0.114220   
                Papule                    5.0  0.103658  0.001994  0.101627   
                Patch                     5.0  0.175139  0.009290  0.162474   
                Plaque                    5.0  0.226478  0.005291  0.219314   
                Pustule                   5.0  0.021828  0.000780  0.020930   
                Scale                     5.0  0.214847  0.003300  0.209803   
                Scar                      5.0  0.216432  0.008709  0.208432   
                Sun damage                5.0  0.244478  0.005457  0.238026   
                Telangiectasia            5.0  0.126707  0.015705  0.112462   
                Thrombosed capillaries    2.0  0.048649  0.000595  0.048228   
inceptionresnet Closed comedo             5.0  0.101433  0.012986  0.085501   
                Cyst                      5.0  0.016848  0.002753  0.014833   
                Dermatoglyph disruption   0.0       NaN       NaN       NaN   
                Leukotrichia              5.0  0.069380  0.045268  0.029256   
                Macule                    5.0  0.101750  0.010782  0.090187   
                Nodule                    5.0  0.030344  0.001927  0.028111   
                Open comedo               5.0  0.113503  0.005933  0.103771   
                Papule                    5.0  0.083119  0.009177  0.073176   
                Patch                     5.0  0.126658  0.006327  0.117639   
                Plaque                    5.0  0.174287  0.005705  0.165214   
                Pustule                   5.0  0.020779  0.004039  0.018102   
                Scale                     5.0  0.174967  0.009000  0.159768   
                Scar                      5.0  0.160492  0.017891  0.143940   
                Sun damage                5.0  0.191852  0.010797  0.173121   
                Telangiectasia            5.0  0.088882  0.014047  0.071830   
                Thrombosed capillaries    1.0  0.002482       NaN  0.002482   
mobilenetv1  

## Per derm

In [139]:
def get_per_derm_metrics(results_path, model_name, images_path, masks_path, classes, columns, derms):
    gradcams_path = results_path / 'visualisation/gradcam' / model_name
    model_path = results_path / model_name
    
    derm_mask_paths = [mask_path for mask_path in masks_path.iterdir() if mask_path.suffix == '.png']
    image_paths = [image_path for image_path in images_path.rglob('*.jpeg')]

    # For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
    # class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
    # As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
    metrics_list = []

    # Set the method used for interpolation when resizing the derm annotations.
    interpolation = Image.NEAREST

    # Compute metrics for each image
    for image_path in image_paths:
        for derm in derms:
            for diagnosis in classes:
                # Check if matching GradCam file exists.
                gradcam_path = gradcams_path / Path(image_path.stem + '_' + diagnosis + '.npy')
                if gradcam_path.is_file():
                    mask_path = masks_path / Path(f'{derm}_{image_path.stem}.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calculate_mask_metrics(gradcam_path, mask_path, interpolation)
                        gradcam_metric_val['gradcam_class'] = diagnosis 
                        gradcam_metric_val['derm'] = derm
                        gradcam_metric_val['filename'] = image_path.stem 
                        metrics_list.append(gradcam_metric_val)
                else:
                    print(f'GradCam file missing for image {image_path}')
                
    return pd.DataFrame.from_records(metrics_list, columns=columns)


In [143]:
def analyse_per_derms_gradcam(
    results_path, 
    model_name, 
    images_path, 
    masks_path, 
    preds_name,
    columns,
    classes,
    derms,
    metrics
):
    per_image_metrics_df = get_per_derm_metrics(results_path, model_name, images_path, masks_path, classes, columns, derms)
    preds_df = pd.read_csv(results_path / preds_name)
    preds_df['filename'] = [filename.split('/')[1].split('.')[0] for filename in preds_df.filename.values]
    preds_df = preds_df.merge(per_image_metrics_df, left_on='filename', right_on='filename')
    # Keep only correct predictions
    preds_df = preds_df[preds_df['actual'] == preds_df['pred']]
    preds_df = preds_df[preds_df['pred_class'] == preds_df['gradcam_class']]
    metrics_dict = {}
    for derm in derms:
        metrics_dict[derm] = {}
        for metric in metrics:
            metrics_dict[derm][metric] = preds_df[preds_df['derm'] == derm][metric].mean()
            
    return metrics_dict


In [146]:
model_info = [
    {
        'model_name': 'efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_',
        'model_architecture': 'efficientnet',
    },
    {
        'model_name': 'inception_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.001_lactivation_85_',
        'model_architecture': 'inception',
    },
    {
        'model_name': 'inceptionresnetv2_r20_s0.25_z0.5_b[0.75, 1.25]_lr0.0001_lblock8_9_ac_',
        'model_architecture': 'inceptionresnet',
    },
    {
        'model_name': 'mobilenetv1_r10_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv_pw_12_relu_',
        'model_architecture': 'mobilenetv1',
    },
    {
        'model_name': 'mobilenetv2_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock_15_add_',
        'model_architecture': 'mobilenetv2',
    },
    {
        'model_name': 'nasnetmobile_r20_s0.25_z0.5_b[0.5, 1.0]_lr0.0001_lnormal_concat_11_',
        'model_architecture': 'nasnetmobile',
    },
    {
        'model_name': 'resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_',
        'model_architecture': 'resnet',
    },
    {
        'model_name': 'resnetv2_r20_s0.25_z0.25_b[0.5, 1.0]_lr0.001_lpost_relu_',
        'model_architecture': 'resnetv2',
    },
    {
        'model_name': 'vgg_r10_s0.0_z0.25_b[0.5, 1.0]_lr0.01_lblock5_pool_',
        'model_architecture': 'vgg',
    },
    {
        'model_name': 'xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_', 
        'model_architecture': 'xception',
    }
]
masks_path = Path('/home/ubuntu/hot-store/dermx_masks/per_derm')
columns = [
    'filename',
    'gradcam_class',
    'derm',
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]
metrics = [
    'iou',
    'dice',
    'precision',
    'recall',
    'negative_predictive_value',
    'specificity'
]

models_performance_dict = {}
for model in model_info:
    for idx in range(5):
        model_results_path = results_path / model['model_architecture']
        model_name = f'{model["model_name"]}{idx}'
        preds_name = f'dermx_{idx}_preds.csv'
        print(model_name)
        model_performance = analyse_per_derms_gradcam(model_results_path, model_name, images_path, masks_path, preds_name, columns, classes, derms, metrics)

        for k, v in model_performance.items():
            models_performance_dict[(model['model_architecture'], model_name, k)] = v

models_performance_df = pd.DataFrame(models_performance_dict).T

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_0


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_1


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_2


...

/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_3


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


xception_r10_s0.25_z0.5_b[0.5, 1.5]_lr0.001_lblock14_sepconv2_act_4


/home/ubuntu/venv/tf2_3/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


In [151]:
models_performance_df.reset_index().groupby(['level_0', 'level_2']).describe()

iou                                          \
                        count      mean       std       min       25%   
level_0         level_2                                                 
efficientnet    derm1     5.0  0.180183  0.008962  0.172147  0.174024   
                derm2     5.0  0.206976  0.006639  0.199934  0.203000   
                derm3     5.0  0.220569  0.014794  0.203390  0.213382   
                derm4     5.0  0.223289  0.008617  0.215216  0.217499   
                derm5     5.0  0.216330  0.004655  0.210022  0.212934   
                derm6     5.0  0.205423  0.005173  0.197133  0.204180   
                derm7     5.0  0.165199  0.005951  0.158387  0.159348   
                derm8     5.0  0.261083  0.004252  0.255101  0.258517   
inception       derm1     5.0  0.186577  0.004040  0.180820  0.185090   
                derm2     5.0  0.217086  0.005250  0.210214  0.213385   
                derm3     5.0  0.259170  0.006997  0.248241  0.257118   
                derm4     5.0  0.224091  0.006450  0.217606  0.219481   
                derm5     5.0  0.234969  0.002874  0.230408  0.234329   
                derm6     5.0  0.207026  0.001572  0.205507  0.206196   
                derm7     5.0  0.184759  0.003632  0.180962  0.182558   
                derm8     5.0  0.283587  0.004649  0.278168  0.280199   
inceptionresnet derm1     5.0  0.160001  0.005300  0.154625  0.157285   
                derm2     5.0  0.171576  0.004258  0.164416  0.171825   
                derm3     5.0  0.186866  0.006597  0.178885  0.183690   
                derm4     5.0  0.183209  0.003426  0.179050  0.181646   
                derm5     5.0  0.170827  0.006334  0.162747  0.166957   
                derm6     5.0  0.155485  0.006917  0.148562  0.149818   
                derm7     5.0  0.170315  0.016146  0.153148  0.160301   
                derm8     5.0  0.213462  0.006788  0.205618  0.207625   
mobilenetv1     derm1     5.0  0.165495  0.005052  0.160590  0.161923   
                derm2     5.0  0.189320  0.007956  0.177566  0.186915   
                derm3     5.0  0.203750  0.003608  0.199325  0.201926   
                derm4     5.0  0.194359  0.009139  0.187600  0.188895   
                derm5     5.0  0.193272  0.007322  0.185756  0.187481   
                derm6     5.0  0.183161  0.008266  0.178058  0.178092   
                derm7     5.0  0.167024  0.003860  0.162687  0.164363   
                derm8     5.0  0.230132  0.004957  0.224759  0.227934   
mobilenetv2     derm1     5.0  0.188071  0.011703  0.173620  0.184039   
                derm2     5.0  0.205521  0.008229  0.193435  0.201348   
                derm3     5.0  0.222380  0.011393  0.212924  0.215102   
                derm4     5.0  0.215101  0.008778  0.207152  0.208330   
                derm5     5.0  0.211614  0.018714  0.191086  0.204564   
                derm6     5.0  0.190748  0.008151  0.176760  0.191177   
                derm7     5.0  0.174528  0.010751  0.163108  0.170069   
                derm8     5.0  0.244425  0.013351  0.231029  0.233707   
nasnetmobile    derm1     5.0  0.201007  0.005990  0.193034  0.196621   
                derm2     5.0  0.223700  0.007304  0.212933  0.221831   
                derm3     5.0  0.263054  0.005939  0.257026  0.259517   
                derm4     5.0  0.238182  0.005523  0.231723  0.234607   
                derm5     5.0  0.260604  0.006742  0.249581  0.260631   
                derm6     5.0  0.193980  0.010170  0.177878  0.190485   
                derm7     5.0  0.188576  0.007087  0.181028  0.185831   
                derm8     5.0  0.298637  0.005547  0.294472  0.294594   
resnet          derm1     5.0  0.140052  0.012166  0.121089  0.138234   
                derm2     5.0  0.156886  0.009942  0.142407  0.151822   
                derm3     5.0  0.177981  0.018960  0.154540  0.167484   
                derm4     5.0  0.147474  0.015698  0.129790  0.140963   
           

### Old

In [2]:
# Set paths

# Path to the GradCam images.
gradcam_main_path = Path('/home/ubuntu/store/efficientnet-final-size/visualisation/gradcam/')

# Set path to the model used to create the visualisations.
model_path = Path('efficientnetb4_0')

# Path to the masks create by the derms for the individual characteristics.
derm_mask_main_path = Path('/home/ubuntu/store/masks/masks_resized')

# Path to the rescaled test images.
test_im_path = Path('/home/ubuntu/store/DermX-test-set/test/')


In [3]:
full_gradcam_path = gradcam_main_path / model_path
full_lime_path = lime_main_path / model_path

In [4]:
classes = [
    "Acne",
    "Actinic keratosis",
    "Psoriasis",
    "Seborrheic dermatitis",
    "Viral warts",
    "Vitiligo"
]

derms = [
    'derm0',
    'derm1',
    'derm2',
    'derm3',
    'derm4',
    'derm5',
    'derm6',
    'derm7',
]

chars = [
    'closed-comedo',
    'cyst',
    'dermatoglyph-disruption',
    'leukotrichia',
    'macule',
    'nodule',
    'open-comedo',
    'papule',
    'patch',
    'plaque',
    'pustule',
    'scale',
    'scar',
    'sun-damage',
    'telangiectasia',
    'thrombosed-capillaries'
]


In [5]:
# Extract the paths to the derm masks. The resulting list is only used for a sanity check.
derm_mask_paths = [p for p in derm_mask_main_path.iterdir() if p.suffix == '.png']
derm_mask_paths[0:5]

[PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043023HB_mihaela_2021-05-27-masks_plaque.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043212HB_adelina_2021-05-27-masks_scale.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043269HB_adelina_2021-05-27-masks_papule.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/293--WatermarkedWyJXYXRlcm1hcmtlZCJd_mihaela_2021-05-27-masks_scale.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/viral-wart-08--WatermarkedWyJXYXRlcm1hcmtlZCJd_oana_2021-05-27-masks_papule.png')]

In [6]:
# Extract the paths to the test images. Only needed for visualization and debugging.
test_img_paths = [p for p in Path(test_im_path).rglob('*.jpeg')]
test_img_paths[0:5]

[PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/017359HB.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/476--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/469--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/3742--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/3753--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg')]

In [7]:
# Metrics supporting probabilistic segmentation maps
fuzzy_and = lambda x,y: np.minimum(x,y)
fuzzy_or = lambda x,y: np.maximum(x,y)
fuzzy_not = lambda x: 1-x

def pixel_metrics_fuzzy(y_true, y_pred):
    """
    Pixel-level metrics of segmentation accuracy following fuzzy logic operators.
    
    :param y_true: numpy.ndarray of reference segmentation, values in [0,1]
    :param y_pred: numpy.ndarray of predicted segmentation, values in [0,1]

    :return: a dictionary encoding the metrics
    """
        
    np.testing.assert_equal(y_true.shape, y_pred.shape, err_msg="Expecting \
    the reference and predicted segmentations to be of the same size.")
    
    # Check the ranges
    np.testing.assert_equal(np.logical_and(y_true >= 0, y_true <= 1).all(), True, err_msg="Expecting \
    the reference segmentations to be in the range 0 to 1.")
    np.testing.assert_equal(np.logical_and(y_pred >= 0, y_pred <= 1).all(), True, err_msg="Expecting \
    the predicted segmentations to be in the range 0 to 1.")
    
    TP = fuzzy_and(y_true, y_pred).sum()
    TN = fuzzy_and(fuzzy_not(y_true), fuzzy_not(y_pred)).sum()
    union = fuzzy_or(y_true, y_pred).sum()
    
    metrics = {}
    
    # Summary metrics
    metrics["iou"] = TP / union
    metrics["dice"] = 2 * TP / ( y_true.sum() + y_pred.sum() ) 
    
    # Positive class metrics
    metrics["precision"] = TP / y_pred.sum()
    metrics["recall"] = TP / y_true.sum()
    
    # Negative class metrics
    metrics["negative_predictive_value"] = TN / fuzzy_not(y_pred).sum()
    metrics["specificity"] = TN / fuzzy_not(y_true).sum()
    
    return metrics

In [8]:
def calc_res(gradcam_image_path, derm_char_mask_path, interpolation_method=Image.NEAREST):
    """
    Calculates the fuzzy logic metrics given the paths to a pair of input images.
    The derm mask is resized to match the size of the gradcam image.
    
    Input:
    - gradcam_image_path: Pathlib path to a gradCam image. The file is assumed to be in .npy format.
    - derm_char_mask_path: Pathlib path to a derm annotation. The file is assumed to be in a format that can
                           be opened by PIL.
    - interpolation_method: String. The method used for interpolation when resizing the derm mask. Options are
                            NEAREST, BOX, BILINEAR, HAMMING, BICUBIC, LANCZOS. Default is NEAREST.    
    """
    
    # Open images.
    gradcam_im = np.load(gradcam_image_path, allow_pickle=True)
    mask_im = Image.open(derm_char_mask_path)
    
    # Resize the derm mask if its size does not match the size of the gradcam image.
    if gradcam_im.shape != mask_im.size[::-1]:
        # Note that resize uses (cols, rows) format, while .shape is in (rows, cols) format.
        mask_im = mask_im.resize((gradcam_im.shape[::-1]), interpolation_method)
    
    # Corvert the derm mask to numpy format and normalize to [0, 1].
    mask_im = np.asarray(mask_im) / 255
  
    res = pixel_metrics_fuzzy(mask_im, gradcam_im)
    return res

In [9]:
# For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
# class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
# As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
rec_dd = lambda: defaultdict(rec_dd)
out_gradcam = rec_dd()
hit_counter = 0

# Set the method used for interpolation when resizing the derm annotations.
interpolation = Image.NEAREST

for p in test_img_paths:
    for c in classes:
        # Check if matching GradCam file exists.
        gc_path = full_gradcam_path / Path(p.stem + '_' + c + '.npy')
        if gc_path.is_file():
            for d in derms:
                for ch in chars:
                    # Check if this derm has created a mask for this characteristic.
                    mask_path = derm_mask_main_path / Path(p.stem + '_' + d + '_2021-05-27-masks_' + ch + '.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calc_res(gc_path, mask_path, interpolation)
                        out_gradcam[p.stem][c][d][ch] = gradcam_metric_val
                        hit_counter += 1
        else:
            print('GradCam or LIME file missing for image: ', p)

print(hit_counter)



19578


In [12]:
# We expect that the value of the hit counter should be equal to the number of derm masks multiplied with the number
# of classes.
if len(derm_mask_paths)*6 != hit_counter:
    print('Oh no, some files were not found. Expected/found: ', len(derm_mask_paths)*6, hit_counter)
else:
    print('GradCam images found for all derm annotations.')


GradCam images found for all derm annotations.


In [13]:
def transform_to_list(nested_dict):
    # Transform the gradcam defaultdict to a list of tuples.
    
    out = []
    for im_name, class_dict in nested_dict.items():
        for class_name, derm_dict in class_dict.items():
            for derm_name, char_dict in derm_dict.items():
                for char_name, metric
                _dict in char_dict.items():
                    tmp = tuple(metric_dict.values())
                    out.append( (im_name, class_name, derm_name, char_name) + tmp )
    return out

In [14]:
gradcam_res_list = transform_to_list(out_gradcam)
len(gradcam_res_list)

19578

## Make output dataFrames/csv files

In [18]:
col_names = ['image_name',
             'visualisation_class',
             'derm',
             'characteristic',
             'iou',
             'dice',
             'precision',
             'recall',
             'negative_predictive_value',
             'specificity'
            ]

In [ ]:
gradcam_df = pd.DataFrame.from_records(gradcam_res_list, columns=col_names)
gradcam_df

In [22]:
# Save dataFrames
model_name = str(model_path)
gradcam_df.to_csv(model_name + "_gradcam_scores.csv")


# Filter

In [123]:
import glob
model_names = glob.glob('/home/ubuntu/store/efficientnet-final-size/*h5')

In [124]:
model_names

['/home/ubuntu/store/efficientnet-final-size/efficientnetb4_4.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_0.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_2.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_3.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_1.h5']

In [125]:
model_name = Path(model_names[1]).stem

In [126]:
def get_table(results_pred):
    results_mean = results_pred.groupby('characteristic').median()
    results_std =  results_pred.groupby('characteristic').mad()
    
    ### Add means
    results_mean.loc['mean'] = results_mean.mean()
    results_std.loc['mean']   = results_mean.mean()
    
    columns = results_mean.columns.to_list()

    table_pred = results_mean
    return table_pred

class_map = {'0' : 'Acne' ,
             '1' : 'Actinic keratosis',
             '2' : 'Psoriasis' ,
             '3' : 'Seborrheic dermatitis',
             '4' : 'Viral warts',
             '5' : 'Vitiligo'}

filefolder = model_name + "_gradcam_scores.csv"
predsfile = '/home/ubuntu/store/efficientnet-final-size/' + model_name + '_preds.csv'

In [127]:
filtered_subjects = pd.read_csv('./include_images_525.csv')
filtered_subjects['image_id'] = filtered_subjects['image_id'].apply(lambda x: Path(x).stem)
filtered_subjects = filtered_subjects.rename(columns={'image_id': 'image_name'})
filtered_subjects = filtered_subjects.drop(columns=['Unnamed: 0'])

In [128]:
df_preds = pd.read_pickle(predsfile)
gradcam_df = pd.read_csv(filefolder)
gradcam_df=gradcam_df.drop(['Unnamed: 0'], axis=1)

In [129]:
df_preds['pred'] =  df_preds['pred'].apply(lambda x: class_map[str(x)])
df_preds['actual'] = df_preds['actual'].apply(lambda x: class_map[str(x)])
df_preds['filenames'] = df_preds['filenames'].apply(lambda x: Path(x).stem)
df_preds = df_preds.rename(columns={'filenames':'image_name'})
df_preds = df_preds.merge(filtered_subjects, on = 'image_name')

In [32]:
result = pd.merge(gradcam_df,df_preds, on = 'image_name')

results_pred = result[result.visualisation_class == result.pred]
table_pred = get_table(results_pred)
# Get it for actual
results_actual = result[result.visualisation_class == result.actual  ]
table_actual = get_table(results_actual)

table_pred.to_pickle('./' + model_name + '_gradcam_visualisation_scores_pred.pkl')
table_actual.to_pickle('./' + model_name + '_gradcam_visualisation_scores_actual.pkl')

results_equal = result[ (result.actual == result.pred) & (result.visualisation_class == result.pred) ]
table_equal = get_table(results_equal)

results_diff = result[ (result.actual != result.pred) & (result.visualisation_class == result.pred) ]
table_diff = get_table(results_diff)

table_equal.to_pickle('./' + model_name + '_gradcam_visualisation_scores_equal.pkl')
table_diff.to_pickle('./' + model_name + '_gradcam_visualisation_scores_diff.pkl')

# Creating Benchmarks

## By concatenation of all models

In [89]:
def lt_format(results_performance, col_n):
    results_mean = results_performance.groupby(col_n).mean()
    results_std = results_performance.groupby(col_n).std()
    
    results_mean.loc['mean'] = results_mean.mean()
    results_std.loc['mean'] = results_std.mean()
    for col in results_performance.columns.to_list():
        results_mean[col] = results_mean[col].apply(lambda x: f'{np.round(x,decimals=2)} $\pm$ ')
        results_std[col] = results_std[col].apply(lambda x: f'{np.round(x,decimals=2)}')
    
    return results_mean + results_std

model_names = glob.glob('/home/ubuntu/store/efficientnet-final-size/*h5')
results_actual = pd.DataFrame()
results_pred = pd.DataFrame()
results_equal = pd.DataFrame()
results_diff = pd.DataFrame()
results_performance = pd.DataFrame()

for mn in sorted(model_names):
    model_name = Path(mn).stem
    performance = pd.read_pickle('./' + model_name + '_performance.pkl')
    actual = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_actual.pkl').loc[:,['recall','specificity','dice']]
    pred = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_pred.pkl').loc[:,['recall','specificity','dice']]
    equal = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_equal.pkl').loc[:,['recall','specificity','dice']]
    diff = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_diff.pkl').loc[:,['recall','specificity','dice']]
    
    performance=performance.drop('mean')
    actual = actual.drop('mean')
    pred = pred.drop('mean')
    equal = equal.drop('mean')
    diff = diff.drop('mean')
    
    results_performance = pd.concat([results_performance, performance])
    results_actual = pd.concat([results_actual, actual])
    results_pred = pd.concat([results_pred, pred]) 
    results_diff = pd.concat([results_diff, diff])

results_performance.index.name = 'label'

table_performance = lt_format(results_performance, 'label')
table_actual = lt_format(results_actual, 'characteristic')

In [90]:
table_performance = table_performance.drop(columns = ['accuracy'])

In [91]:
table_performance

,precision,f1-score,specificity
label,,,
acne,0.77 $\pm$ 0.16,0.65 $\pm$ 0.32,0.97 $\pm$ 0.02
actinic_keratosis,0.74 $\pm$ 0.08,0.55 $\pm$ 0.11,0.96 $\pm$ 0.02
psoriasis_no_pustular,0.43 $\pm$ 0.14,0.57 $\pm$ 0.12,0.67 $\pm$ 0.23
seborrheic_dermatitis,0.62 $\pm$ 0.08,0.45 $\pm$ 0.22,0.95 $\pm$ 0.03
vitiligo,0.0 $\pm$ 0.01,0.0 $\pm$ 0.01,0.9 $\pm$ 0.03
wart,0.08 $\pm$ 0.04,0.07 $\pm$ 0.04,0.86 $\pm$ 0.04
mean,0.44 $\pm$ 0.08,0.38 $\pm$ 0.14,0.89 $\pm$ 0.06


In [87]:
print(table_performance.to_latex())

\begin{tabular}{llll}
\toprule
{} &        precision &         f1-score &      specificity \\
label                 &                  &                  &                  \\
\midrule
acne                  &  0.77 \$\textbackslash pm\$ 0.16 &  0.65 \$\textbackslash pm\$ 0.32 &  0.97 \$\textbackslash pm\$ 0.02 \\
actinic\_keratosis     &  0.74 \$\textbackslash pm\$ 0.08 &  0.55 \$\textbackslash pm\$ 0.11 &  0.96 \$\textbackslash pm\$ 0.02 \\
psoriasis\_no\_pustular &  0.43 \$\textbackslash pm\$ 0.14 &  0.57 \$\textbackslash pm\$ 0.12 &  0.67 \$\textbackslash pm\$ 0.23 \\
seborrheic\_dermatitis &  0.62 \$\textbackslash pm\$ 0.08 &  0.45 \$\textbackslash pm\$ 0.22 &  0.95 \$\textbackslash pm\$ 0.03 \\
vitiligo              &   0.0 \$\textbackslash pm\$ 0.01 &   0.0 \$\textbackslash pm\$ 0.01 &   0.9 \$\textbackslash pm\$ 0.03 \\
wart                  &  0.08 \$\textbackslash pm\$ 0.04 &  0.07 \$\textbackslash pm\$ 0.04 &  0.86 \$\textbackslash pm\$ 0.04 \\
mean                  &  0.44 \

In [39]:
table_actual

,recall,specificity,dice
characteristic,,,
closed-comedo,0.21 $\pm$ 0.1,0.83 $\pm$ 0.04,0.08 $\pm$ 0.03
cyst,0.2 $\pm$ 0.14,0.85 $\pm$ 0.03,0.02 $\pm$ 0.01
dermatoglyph-disruption,0.09 $\pm$ 0.05,0.79 $\pm$ 0.06,0.05 $\pm$ 0.02
leukotrichia,0.14 $\pm$ 0.1,0.82 $\pm$ 0.04,0.07 $\pm$ 0.03
macule,0.27 $\pm$ 0.07,0.8 $\pm$ 0.03,0.09 $\pm$ 0.03
nodule,0.2 $\pm$ 0.08,0.82 $\pm$ 0.03,0.03 $\pm$ 0.01
open-comedo,0.19 $\pm$ 0.1,0.83 $\pm$ 0.04,0.08 $\pm$ 0.04
papule,0.23 $\pm$ 0.07,0.8 $\pm$ 0.03,0.07 $\pm$ 0.01
patch,0.28 $\pm$ 0.06,0.8 $\pm$ 0.03,0.21 $\pm$ 0.04


In [40]:
print(table_actual[['dice','recall','specificity']].to_latex())

\begin{tabular}{llll}
\toprule
{} &             dice &           recall &      specificity \\
characteristic          &                  &                  &                  \\
\midrule
closed-comedo           &  0.08 \$\textbackslash pm\$ 0.03 &   0.21 \$\textbackslash pm\$ 0.1 &  0.83 \$\textbackslash pm\$ 0.04 \\
cyst                    &  0.02 \$\textbackslash pm\$ 0.01 &   0.2 \$\textbackslash pm\$ 0.14 &  0.85 \$\textbackslash pm\$ 0.03 \\
dermatoglyph-disruption &  0.05 \$\textbackslash pm\$ 0.02 &  0.09 \$\textbackslash pm\$ 0.05 &  0.79 \$\textbackslash pm\$ 0.06 \\
leukotrichia            &  0.07 \$\textbackslash pm\$ 0.03 &   0.14 \$\textbackslash pm\$ 0.1 &  0.82 \$\textbackslash pm\$ 0.04 \\
macule                  &  0.09 \$\textbackslash pm\$ 0.03 &  0.27 \$\textbackslash pm\$ 0.07 &   0.8 \$\textbackslash pm\$ 0.03 \\
nodule                  &  0.03 \$\textbackslash pm\$ 0.01 &   0.2 \$\textbackslash pm\$ 0.08 &  0.82 \$\textbackslash pm\$ 0.03 \\
open-comedo          